In [25]:
using DynamicPolynomials, LinearAlgebra

@ncpolyvar x1 x2 y1 y2 

var=Vector{PolyVar{false}}([x1;x2;y1;y2])

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end

function get_star_alg(f)
    mom=monomials(f)
    coe=coefficients(f)
    mom=star_algebra.(mom)
    return dot(mom,coe)
end

u = x1*y1 + x1*y2 + x2*y1 - x2*y2

f =-(u + get_star_alg(u))/2

g=Vector{Polynomial{false,Float64}}([])

h=[1.0-x1^2;1-x2^2;1-y1^2;1-y2^2;x1*y1-y1*x1;x2*y1-y1*x2;x1*y2-y2*x1;x2*y2-y2*x2];

In [26]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(var,f,g,h);

In [27]:
println("Number of variables: n=",n)
println("Number of inequality constraints: m=",m)
println("Number of equality constraints: l=",l)

Number of variables: n=4
Number of inequality constraints: m=0
Number of equality constraints: l=8


In [28]:
k=2

println("Relaxation order: k=",k)

Relaxation order: k=2


In [29]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                                 maxit=Int64(1e6),tol=1e-3,
                                 use_eqcons_to_get_constant_trace=true,
                                 check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 21.0
  Number of blocks: omega=1
  Size of the largest block: s^max=21
  Number of equality trace constraints: zeta=141
Modeling time:
  0.195514 seconds (221.73 k allocations: 12.374 MiB)
iter=1   val=-14.849242404917499   gap=2.241566876217509e-16   feas=36.76360292463186
iter=2   val=-4.949747468305834   gap=0.6920622242586193   feas=12.980743896091104
iter=4   val=-5.934974315774804   gap=0.23719950679988197   feas=9.24936839190288
iter=8   val=-1.6486039766041125   gap=0.5662580107054244   feas=3.592620306715505
iter=16   val=-3.274217984830528   gap=0.3523122991075588   feas=2.4400740775332324
iter=32   val=-2.248420279668945   gap=0.29155749189377705   feas=1.1889146796047911
iter=64   val=-2.5194388406734087   gap=0.18580276756653458   feas=0.6774400265017426
iter=128   val=-2.6453779121057046   gap=0.06364504995708592   feas=0.39423985435391484
iter=256   val=-2.8147704546260517   gap=0.11444398325059893   feas=

-2.8280440623060654

In [30]:
using NCTSSOS

@time ctpNCPOP.nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],
    n,k,[dg;dh],numeq=l,reducebasis=false,obj="eigen",merge=false,TS=false,QUIET=false);

------------------------------------------------------
The sizes of blocks:
[21]
[1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 191             
  Cones                  : 0               
  Scalar variables       : 121             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - numb